In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer

# [Tutorial](https://hackernoon.com/want-a-complete-guide-for-xgboost-model-in-python-using-scikit-learn-sc11f31bq)

## Load the Data 

In [2]:
df = pd.read_csv("csv/book_info_complete.csv")

### Keep Description and CSM_Rating

In [3]:
df = df.loc[:, ['description', "csm_rating"]]

In [4]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=999)
for train_index, test_index in split.split(df, df['csm_rating']):
    train_data= df.loc[train_index]
    test_data = df.loc[test_index]

In [5]:
train_data = train_data.reset_index()

In [6]:
test_data.shape

(1164, 2)

In [7]:
#instantiate CountVectorizer()
cv=CountVectorizer()

# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(train_data['description'])

In [8]:
X_x = pd.DataFrame(word_count_vector.todense(), columns=cv.get_feature_names())

In [9]:
X_x.shape[0]

4652

In [10]:
train_df = pd.concat([X_x, train_data.loc[:, "csm_rating"]], axis=1, ignore_index=True)
#result = pd.concat([df1, s1], axis=1)
train_df.shape

(4652, 5619)

In [8]:
data_dmatrix = xgb.DMatrix(data=X_x,label=X_y)

In [9]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)